<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Notes:" data-toc-modified-id="Notes:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notes:</a></span></li><li><span><a href="#Prepare-data_info_file" data-toc-modified-id="Prepare-data_info_file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Prepare data_info_file</a></span><ul class="toc-item"><li><span><a href="#add-mESC-data" data-toc-modified-id="add-mESC-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>add mESC data</a></span></li></ul></li><li><span><a href="#Prepare-query_region-bed-from-DMR-list" data-toc-modified-id="Prepare-query_region-bed-from-DMR-list-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prepare query_region bed from DMR list</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Qsub-dir" data-toc-modified-id="Qsub-dir-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Qsub dir</a></span><ul class="toc-item"><li><span><a href="#REPTILE-Train" data-toc-modified-id="REPTILE-Train-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>REPTILE Train</a></span></li><li><span><a href="#REPTILE-Pred" data-toc-modified-id="REPTILE-Pred-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>REPTILE Pred</a></span></li></ul></li><li><span><a href="#REPTILE-Compute-Score" data-toc-modified-id="REPTILE-Compute-Score-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>REPTILE Compute Score</a></span></li><li><span><a href="#Make-Score-Track" data-toc-modified-id="Make-Score-Track-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Make Score Track</a></span><ul class="toc-item"><li><span><a href="#The-region-bed-generated-by-REPTILE-is-overlapped" data-toc-modified-id="The-region-bed-generated-by-REPTILE-is-overlapped-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>The region bed generated by REPTILE is overlapped</a></span></li></ul></li><li><span><a href="#REPTILE-call-enhancer" data-toc-modified-id="REPTILE-call-enhancer-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>REPTILE call enhancer</a></span></li></ul></div>

# REPTILE Pipeline

## Packages
R=3.2
python=3.5
numpy=1.10
pandas=0.17

## Notes:
- Be careful about sample names, R change a lot of things... Use A-Za-z0-9 should be safe... But do not use number
- Do not use number in DMR id
- Some steps need large MEM, e.g. prepare_prediction, predict_score and sort score bed


![REPTILE FILE FORMAT](https://raw.githubusercontent.com/yupenghe/misc/master/REPTILE/file_format.png)

In [47]:
import pandas as pd
import pathlib

reptile_ref_dir = '/gale/netapp/home/hanliu/ref/inhouse/REPTILE_training/'
reptile_ref_dir = pathlib.Path(reptile_ref_dir)

reptile_code_dir = '/gale/netapp/home/hanliu/pkg/REPTILE/bin/'
reptile_code_dir = pathlib.Path(reptile_code_dir)

cell_tidy_data_path = \
'/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'

## Prepare data_info_file

In [3]:
cell_tidy_data = pd.read_msgpack(cell_tidy_data_path)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
path_records = []

mc_bw_dir = pathlib.Path('/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_brain/MultiLevelCluster/BW/SubType/')
mc_bw_list = list(mc_bw_dir.glob('*CGN*rate*bw'))

for path in mc_bw_list:
    sample = path.name.split('.')[1]
    path_records.append({'sample': sample, 'mark': 'mCG', 'bw_file': path})
    
atac_bw_dir = pathlib.Path('/home/hanliu/project/mouse_rostral_brain/ATAC/remerge/mCSubType/')
atac_bw_list = list(atac_bw_dir.glob('*bw'))

for path in atac_bw_list:
    sample = path.name.split('.')[0]
    path_records.append({'sample': sample, 'mark': 'ATAC', 'bw_file': path})
data_info_df = pd.DataFrame(path_records)
# data_info_df['sample'] = '' + data_info_df['sample']

r_shit = data_info_df['sample'].str.replace('-', '.').str.replace('_', '.')
name_map = {a:b for a, b in zip(data_info_df['sample'], r_shit)}
data_info_df['sample'] = r_shit

import joblib
joblib.dump(name_map, 'NameMap.lib')

assert len(data_info_df['sample'].value_counts().unique()) == 1

data_info_df.shape

(322, 3)

### add mESC data


In [38]:
path_records = []
path_records.append({'sample': 'mESC', 'mark': 'mCG', 'bw_file': f'{reptile_ref_dir}/mESC_Meth.bw'})
path_records.append({'sample': 'mESC', 'mark': 'ATAC', 'bw_file': f'{reptile_ref_dir}/mESC_ATAC.bw'})
mesc_df = pd.DataFrame(path_records)
data_info_df = pd.concat([data_info_df, mesc_df], ignore_index=True)
# r do not support '-', r will change the col name silently... F**K R
data_info_df['sample'] = data_info_df['sample'].str.replace('-', '')
data_info_df.shape

(324, 3)

In [39]:
data_info_df.to_csv('data_info_file.tsv', index=None, sep='\t')
data_info_df.head()

,sample,mark,bw_file
0,ASC.str.hpf,mCG,/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_bra...
1,PAL.Inh.Igdcc3,mCG,/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_bra...
2,CA1.Ak5,mCG,/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_bra...
3,ASC.mid,mCG,/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_bra...
4,OPC.opc.large,mCG,/gale/ddn/cemba/CEMBA/hanliu/mouse_rostral_bra...


In [40]:
data_info_df['mark'].value_counts()

mCG     162
ATAC    162
Name: mark, dtype: int64

## Prepare query_region bed from DMR list
- No filter in this step

In [42]:
dmr_path = '/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalDMR.nofilter.bed'

![REPTILE PIPELINE](https://raw.githubusercontent.com/yupenghe/misc/master/REPTILE/workflow.png)

## Training

In [43]:
data_info_file=pathlib.Path('./data_info_file.tsv').absolute()
dmr_file=pathlib.Path('./dmr_query_region.bed').absolute()
genome_region_file = reptile_ref_dir / 'mm10_w2kb_s100bp.bed'
training_region= reptile_ref_dir / 'mESC_region_for_train.bed'
training_state= reptile_ref_dir / 'mESC_region_for_train_label.tsv'

In [44]:
model_dir = pathlib.Path('model').absolute()
model_dir.mkdir(exist_ok=True)

## Qsub dir

In [45]:
qsub_dir = pathlib.Path('qsub').absolute()
qsub_dir.mkdir(exist_ok=True)

### REPTILE Train

In [46]:
qsub_template = """
#!/bin/bash
#$ -N {name}
#$ -V
#$ -pe smp 12
#$ -l h_vmem=5G
#$ -l h_rt=99:99:99
#$ -l s_rt=99:99:99
#$ -e {qsub_dir}/{name}.error.log
#$ -o {qsub_dir}/{name}.output.log

{cmd}
"""

name = 'reptile_prepare_training'
cmd = f'{reptile_code_dir}/REPTILE_preprocess.py '\
       f'{data_info_file} {training_region} '\
       f'{model_dir}/mESC_training -p 10 -d {dmr_file}'

with open(qsub_dir / f'{name}.sh', 'w') as f:
    f.write(qsub_template.format(name=name, cmd=cmd, qsub_dir=qsub_dir))
    
name = 'reptile_training'
cmd = f'{reptile_code_dir}/REPTILE_train.R '\
      f'-i {data_info_file} '\
      f'-d {model_dir}/mESC_training.DMR_with_epimark.tsv '\
      f'-a {model_dir}/mESC_training.region_with_epimark.tsv '\
      f'-l {training_state} -s mESC '\
      f'-o {model_dir}/REPTILE_model'

with open(qsub_dir / f'{name}.sh', 'w') as f:
    f.write(qsub_template.format(name=name, cmd=cmd, qsub_dir=qsub_dir))

### REPTILE Pred

In [23]:
# cmd2 needs larger memory
cmd = f'{reptile_code_dir}/REPTILE_preprocess.py {data_info_file} '\
       f'{genome_region_file} '\
       f'-d {dmr_file} {model_dir}/mm10_w2kb_s100bp '\
       f'-p 25 -n 30 -g'

name = 'reptile_prepare_predict'

qsub_template = """
#!/bin/bash
#$ -N {name}
#$ -V
#$ -pe smp 40
#$ -l h_vmem=6G
#$ -l h_rt=99:99:99
#$ -l s_rt=99:99:99
#$ -e {qsub_dir}/{name}.error.log
#$ -o {qsub_dir}/{name}.output.log

{cmd}
"""

with open(qsub_dir / f'{name}.sh', 'w') as f:
    f.write(qsub_template.format(name=name, cmd=cmd, qsub_dir=qsub_dir))

## REPTILE Compute Score

In [24]:
data_info_file_df = pd.read_table(data_info_file)

In [25]:
result_dir = pathlib.Path('results').absolute()
result_dir.mkdir(exist_ok=True)

total_cmd = ''
for sample in data_info_file_df['sample'].unique():
    cmd = f'{reptile_code_dir}/REPTILE_compute_score.R -i {data_info_file} '\
          f'-m {model_dir}/REPTILE_model.reptile '\
          f'-d {model_dir}/mm10_w2kb_s100bp.DMR_with_epimark '\
          f'-a {model_dir}/mm10_w2kb_s100bp.region_with_epimark '\
          f'-s {sample} -o {result_dir}/{sample}_pred -p 16 -n 30'
    total_cmd += f'{cmd}\n'


name = 'reptile_predict'
qsub_template = """
{cmd}
"""
with open(qsub_dir / f'{name}.sh', 'w') as f:
    f.write(qsub_template.format(name=name, cmd=total_cmd, qsub_dir=qsub_dir))

## Make Score Track

### The region bed generated by REPTILE is overlapped
- Make it nonoverlap
- Remove zero
- Sort

In [ ]:
region_score_list = list(result_dir.glob('*.R.bed'))
for path in region_score_list:
    df = pd.read_csv(path, header=None, sep='\t', usecols=[0, 1, 2, 4])
    df[1] = df[1] + 950
    df[2] = df[2] - 950
    df = df[((df[2] - df[1]) == 100) & (df[4] != 0)].copy()
    df.sort_values(by=[0, 1], inplace=True)
    df.to_csv(str(path) + 'sorted', index=None, header=None, sep='\t')

In [ ]:
import subprocess
sorted_list = list(result_dir.glob('*.R.bedsorted'))

for path in sorted_list:
    subprocess.run(['bedGraphToBigWig', 
                    str(path), 
                    '/gale/netapp/home/hanliu/ref/mouse/genome/fasta/raw/mm10.chrom.sizes', 
                    str(path)[:-10] + '.bw'], check=True)
    print(path)

## REPTILE call enhancer

In [19]:
result_dir = pathlib.Path('results').absolute()
result_dir.mkdir(exist_ok=True)

total_cmd = ''
for sample in data_info_file_df['sample'].unique():
    cmd = f'nohup {reptile_code_dir}/REPTILE_call_enhancer.py '\
      f'-d {result_dir}/{sample}_pred.DMR.bed '\
      f'-o {result_dir}/REPTILE_pred_{sample}.bed '\
      f'{result_dir}/{sample}_pred.R.bed -p 0.5 &'
    total_cmd += f'{cmd}\n'

name = 'reptile_call_enhancer'
qsub_template = """
{cmd}
"""
with open(qsub_dir / f'{name}.sh', 'w') as f:
    f.write(qsub_template.format(name=name, cmd=total_cmd, qsub_dir=qsub_dir))